In [1]:
# download nltk stopwords
import nltk
nltk.download('stopwords')

# Install a particular version of `google-cloud-storage` because (oddly enough) 
# the  version on Colab and GCP is old. A dependency error below is okay.
!pip install -q google-cloud-storage==1.43.0

# authenticate below for Google Storage access as needed
from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/drive/')
import sys
sys.path.append('/content/drive/MyDrive/Information Retrieval/Project/information-retrieval-final-project/colab_cache')
%cd /content/drive/MyDrive/Information Retrieval/Project/information-retrieval-final-project/colab_cache

# install ngrok to emulate public IP / address
!wget -N https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip

# TODO: sign up for an ngrok account
# then put your ngrok token below, uncomment, and execute
!./ngrok authtoken 2K5pCuAwvo0Kajtz8DA5HwCRr7f_2Rssap33tUn3qcHuxFDBY

# install a ngrok python package and a version of flask that works with it in 
# colab
!pip -q install flask-ngrok
!pip -q install flask==0.12.2
# !pip -q install flask_restful

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 KB 3.2 MB/s eta 0:00:00
Mounted at /content/drive/
/content/drive/MyDrive/Information Retrieval/Project/information-retrieval-final-project/colab_cache
for details.

--2023-01-09 21:32:22--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 18.205.222.128, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  30.2MB/s    in 0.4s    

2023-01-09 21:32:23 (30.2 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
/bin/bash: ./ngrok: Permission denied
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 KB 3.4 MB/s eta 0:00:00


# Run the app

In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 15.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=51c98d2579911975831bfe5b43e1f1db9a89ad23ba2d5eafb9c85cf8355ed7ee
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [3]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import numpy as np
from collections import defaultdict, Counter
import pandas as pd
from inverted_index_colab import *
from pyspark.sql import SparkSession
import colab_search_frontend as se
import time


spark = SparkSession.builder.appName("colab_front").getOrCreate()
id_title_pagerank_pageviews = spark.read.format("csv").option("compression", "gzip").load("colab_pagerank.csv.gz",header = True)

se.load_data(id_title_pagerank_pageviews) # this func is implemented in search frontend in order to set 
                                   # up an all the variables search functions might need.

"""
def load_data():
    # load the three indexes
    idx_body = InvertedIndex.read_index('.', 'body_index')
    idx_title = InvertedIndex.read_index('.', 'title_index')
    idx_anchor = InvertedIndex.read_index('.', 'anchor_index')

    # declare the variables we will need in other functions (search) as global
    global body_words_pls
    global title_words_pls
    global anchor_words_pls
    global id_pagerank_title_df

    # make a dictionary such as {word1: pls1, word2: pls2 ...}
    body_words_pls = dict(zip(*zip(*idx_body.posting_lists_iter())))
    title_words_pls = dict(zip(*zip(*idx_title.posting_lists_iter())))
    anchor_words_pls = dict(zip(*zip(*idx_anchor.posting_lists_iter())))


    # load the csv that contains all of the pageranks, titles, and pageviews as a spark DF
    spark = SparkSession.builder.appName("Example").getOrCreate()
    id_pagerank_title_df = spark.read.csv("colab_pagerank.csv", header=True, inferSchema=True)
    spark.stop()
"""

#id_title_pagerank_pageviews.show()


start = time.time()
res = se.search_title()
res2 = se.search_anchor()
#res3 = se.search_body()
end = time.time()
print(end - start)
spark.stop()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The word afrikaans does NOT appear in our index
The word al-qaeda does NOT appear in our index
The word andorra does NOT appear in our index
31.501163721084595


In [ ]:
#global id_pagerank_title_df

# idx_body = InvertedIndex.read_index('.', 'body_index')
# idx_title = InvertedIndex.read_index('.', 'title_index')
# idx_anchor = InvertedIndex.read_index('.', 'anchor_index')

# id_pagerank_title_df = spark.read.csv("colab_pagerank.csv", header=True, inferSchema=True)
# id_pagerank_title_df = pd.read_csv("colab_pagerank.csv", index_col=0)

# body_words_pls = dict(zip(*zip(*idx_body.posting_lists_iter())))
# title_words_pls = dict(zip(*zip(*idx_title.posting_lists_iter())))
# anchor_words_pls = dict(zip(*zip(*idx_anchor.posting_lists_iter())))

In [11]:
# uncomment the code below and execute to reload the module when you make 
# changes to search_frontend.py (after you upload again).
import importlib
importlib.reload(se)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<module 'colab_search_frontend' from '/content/drive/MyDrive/Information Retrieval/Project/information-retrieval-final-project/colab_cache/colab_search_frontend.py'>

In [ ]:
from flask_ngrok import run_with_ngrok
run_with_ngrok(se.app) 
se.app.run()

 * Serving Flask app "search_frontend" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://be08-35-196-42-90.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


# Testing your app

Once your app is running you can query it. You can simply do that by navigating to the URL that ngrok gave you above or through code in a different python session. For example, once the frontend app is running, you can navigate to:
http://YOUR_SERVER_DOMAIN/search?query=hello+world where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io, which is printed above in Colab or that is your external IP on GCP.

The code below shows how to issue a query from python. This is also how our testing code will issue queries to your search engine, so make sure to test your search engine this way after you deploy it to GCP and before submission. Command line instructions for deploying your search engine to GCP are available at `run_frontend_in_gcp.sh`. Note that we will not only issue training queries to your search engine, but also test queries, i.e. queries that you've never seen before.

In [ ]:
import json

with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [ ]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [ ]:
import requests
from time import time
# url = 'http://35.232.59.3:8080'
# place the domain you got from ngrok or GCP IP below. 
url = 'http://XXXX-XX-XX-XX-XX.ngrok.io'

qs_res = []
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search', {'query': q}, timeout=35)
    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      ap = average_precision(true_wids, pred_wids)
  except:
    pass
  
  qs_res.append((q, duration, ap))